In [3]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

import keras # broken for keras >= 2.0, use 1.2.2
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import Flatten, Dense, Activation, Reshape

from utils import load_weights, Box, yolo_net_out_to_car_boxes, draw_box

Imageio: 'ffmpeg-win32-v3.2.4.exe' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-win32-v3.2.4.exe (34.1 MB)
Downloading: 8192/35749888 bytes (0.057344/35749888 bytes (0.2%237568/35749888 bytes (0.7450560/35749888 bytes (1.3679936/35749888 bytes (1.9909312/35749888 bytes (2.51073152/35749888 bytes (3.0%1269760/35749888 bytes (3.6%1695744/35749888 bytes (4.7%1966080/35749888 bytes (5.5%2269184/35749888 bytes (6.3%2498560/35749888 bytes (7.0%2719744/35749888 bytes (7.6%2924544/35749888 bytes (8.2%3129344/35749888 bytes (8.8%3358720/35749888 bytes (9.4%3653632/35749888 bytes (10.23842048/35749888 bytes (10.74120576/35749888 bytes (11.54399104/35749888 bytes (12.34595712/35749888 bytes (12.94792320/35749888 bytes (13.45005312/35749888 bytes (14.05316608/35749888 bytes (14.95529600/35749888 bytes (15.55824512/35749888 bytes (16.36103040/35749888 bytes (17.16316032/35749888 bytes (17.76627328/35749

In [4]:
keras.backend.set_image_dim_ordering('th')

In [5]:
model = Sequential()
model.add(Convolution2D(16, 3, 3,input_shape=(3,448,448),border_mode='same',subsample=(1,1)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(32,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),border_mode='valid'))
model.add(Convolution2D(64,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),border_mode='valid'))
model.add(Convolution2D(128,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),border_mode='valid'))
model.add(Convolution2D(256,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),border_mode='valid'))
model.add(Convolution2D(512,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),border_mode='valid'))
model.add(Convolution2D(1024,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Convolution2D(1024,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Convolution2D(1024,3,3 ,border_mode='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(Flatten())
model.add(Dense(256))
model.add(Dense(4096))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(1470))

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 448, 448)      448       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 448, 448)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 224, 224)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 224, 224)      4640      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32, 224, 224)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 112, 112)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 112, 112)      18496     
__________

In [14]:
load_weights(model,'./yolo-tiny.weights')

In [15]:
imagePath = './test_images/test1.jpg'
image = plt.imread(imagePath)
image_crop = image[300:650,500:,:]
resized = cv2.resize(image_crop,(448,448))

In [16]:
batch = np.transpose(resized,(2,0,1))
batch = 2*(batch/255.) - 1
batch = np.expand_dims(batch, axis=0)
out = model.predict(batch)